# MOFs LM Probing for Topology

In [1]:
from baseline import PromptSet

from tqdm.auto import tqdm
from nltk.corpus import stopwords

import pandas as pd

from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline, logging

logging.set_verbosity_error()  # avoid irritating transformers warnings

## Read in the Triples

In [2]:
file = "../data/mof_topology.csv"

In [3]:
df = pd.read_csv(file)
df.head()

,SubjectEntity,Relation,ObjectEntity
0,MOF-CJ3,hasTopology,sab
1,"catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-t...",hasTopology,tbo
2,NJU-Bai9,hasTopology,pbz
3,MOF-1,hasTopology,pcu
4,MOF-2,hasTopology,soc


In [4]:
df.shape

(748, 3)

In [5]:
df.ObjectEntity.nunique()

152

## Probing BERT

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
model = AutoModelForMaskedLM.from_pretrained("bert-large-cased")

In [7]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [8]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Create Prompts

In [9]:
def create_topology_prompt1(sub, mask):
    return f"{sub} is an MOF. {sub} has the topology {mask}."
    
def create_topology_prompt2(sub, mask):
    return f"{sub} is an MOF. {sub} has an topology code {mask}."

def create_topology_prompt3(sub, mask):
    return f"{sub} is an MOF. {sub} has an tolology RCSR code {mask}."

def create_topology_prompt4(sub, mask):
    return f"{sub} is an MOF. The topology of {sub} is {mask}."
    
def create_topology_prompt5(sub, mask):
    return f"{sub} is an MOF. {sub} has the three letter symbol {mask}."
    
def create_topology_prompt6(sub, mask):
    return f"{sub} is an metal organic framework. {sub}'s topology is {mask}." 
    
def create_topology_prompt7(sub, mask):
    return f"{sub} is an MOF. {sub} is an type of {mask} topology."
    
def create_topology_prompt8(sub, mask):
    return f"{sub} is an MOF. {sub} is in the class of {mask} topology."
    
def create_topology_prompt9(sub, mask):
    return f"{sub} has SBUs and organic linkers. The topology of {sub} is {mask}."
    
def create_topology_prompt10(sub, mask):
    return f"{sub} is an MOF. The three letter {mask} describes {sub} topology."
    
def create_topology_prompt11(sub, mask):
    return f"{sub} is a MOF. The RCSR code of {sub} is {mask}."
    
def create_topology_prompt12(sub, mask):
    return f"As a MOF, {sub} has a topological structure {mask}."

In [10]:
processors = [create_topology_prompt1, create_topology_prompt2,
              create_topology_prompt3, create_topology_prompt4,
              create_topology_prompt5, create_topology_prompt6,
              create_topology_prompt7, create_topology_prompt8,
              create_topology_prompt9, create_topology_prompt10,
              create_topology_prompt11, create_topology_prompt12
             ]

## Check Whether the Predicted Tokens Match

In [11]:
# Compute a column indicating whether the predictedToken in the ObjectEntity list
def hasPredicted(row):
    token = row['predictedToken']
    objectEntities = row['ObjectEntity']
    #print(objectEntities)
    if pd.notna(objectEntities):
        types = objectEntities.split(",") # get individual types
        for tp in types: # for each type
            type_words = tp.lower().split() # get the individual words in the type
            if token.lower() in type_words: # check if a type word was predicted
                return 1
       
    return 0

In [63]:
# test what the function does
def hasPredicted_demo(objectEntities, token):
    #print(objectEntities)
    if pd.notna(objectEntities):
        types = objectEntities.split(",") # get individual types
        for tp in types: # for each type
            type_words = tp.lower().split() # get the individual words in the type
            print(type_words)
            if token.lower() in type_words: # check if a type word was predicted
                return 1
       
    return 0

In [64]:
hasPredicted_demo('tpc,ptc', 'ptc')

['tpc']
['ptc']


1

## Test Small

In [21]:
processors = [create_topology_prompt1, create_topology_prompt2,
              create_topology_prompt3]

In [22]:
processors[0]('MOF-1', '[MASK]')

'MOF-1 has the topology symbol [MASK].'

In [23]:
mofs_sub =df[:10]

In [53]:
prompts = PromptSet([processors[0](row.SubjectEntity, mask_token) for _, row in mofs_sub.iterrows()])

In [54]:
prompts.__getitem__(9)

'DUT-63 is an MOF. DUT-63 has the topology [MASK].'

In [55]:
for i in prompts:
    print(i)

MOF-CJ3 is an MOF. MOF-CJ3 has the topology [MASK].
catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-triyltribenzoato)-dodeca-aqua-dodeca-copper dimethylsulfoxide solvate hydrate)|PCN-6|Teaching Subset: MOFs is an MOF. catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-triyltribenzoato)-dodeca-aqua-dodeca-copper dimethylsulfoxide solvate hydrate)|PCN-6|Teaching Subset: MOFs has the topology [MASK].
NJU-Bai9 is an MOF. NJU-Bai9 has the topology [MASK].
MOF-1 is an MOF. MOF-1 has the topology [MASK].
MOF-2 is an MOF. MOF-2 has the topology [MASK].
URMOF-1 is an MOF. URMOF-1 has the topology [MASK].
URMOF-2 is an MOF. URMOF-2 has the topology [MASK].
UMCM-151 is an MOF. UMCM-151 has the topology [MASK].
UMCM-152 is an MOF. UMCM-152 has the topology [MASK].
DUT-63 is an MOF. DUT-63 has the topology [MASK].


In [57]:
# Run the model
outputs = []
for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
    outputs.append(out)

  0%|          | 0/10 [00:00<?, ?it/s]

In [58]:
outputs

[[{'score': 0.23990175127983093,
   'token': 1378,
   'token_str': 'following',
   'sequence': 'MOF - CJ3 is an MOF. MOF - CJ3 has the topology following.'},
  {'score': 0.13317494094371796,
   'token': 1104,
   'token_str': 'of',
   'sequence': 'MOF - CJ3 is an MOF. MOF - CJ3 has the topology of.'},
  {'score': 0.11866284161806107,
   'token': 131,
   'token_str': ':',
   'sequence': 'MOF - CJ3 is an MOF. MOF - CJ3 has the topology :.'},
  {'score': 0.10380640625953674,
   'token': 3226,
   'token_str': 'follows',
   'sequence': 'MOF - CJ3 is an MOF. MOF - CJ3 has the topology follows.'},
  {'score': 0.03784811869263649,
   'token': 5777,
   'token_str': 'domain',
   'sequence': 'MOF - CJ3 is an MOF. MOF - CJ3 has the topology domain.'},
  {'score': 0.01953132078051567,
   'token': 1114,
   'token_str': 'with',
   'sequence': 'MOF - CJ3 is an MOF. MOF - CJ3 has the topology with.'},
  {'score': 0.018506860360503197,
   'token': 117,
   'token_str': ',',
   'sequence': 'MOF - CJ3 is an

In [59]:
results = []
for idx, row in mofs_sub.iterrows():
    for output in outputs[idx]:
        record = {}
        record['SubjectEntity'] = row['SubjectEntity']
        record['Relation'] = row['Relation']
        record['ObjectEntity'] = row['ObjectEntity']
        record['prompt']= prompts[idx]
        record['predictedScore'] = output['score']
        record['predictedToken'] = output['token_str']
        results.append(record)

In [60]:
predicted_df = pd.DataFrame(results)
predicted_df

,SubjectEntity,Relation,ObjectEntity,prompt,predictedScore,predictedToken
0,MOF-CJ3,hasTopology,sab,MOF-CJ3 is an MOF. MOF-CJ3 has the topology [M...,0.239902,following
1,MOF-CJ3,hasTopology,sab,MOF-CJ3 is an MOF. MOF-CJ3 has the topology [M...,0.133175,of
2,MOF-CJ3,hasTopology,sab,MOF-CJ3 is an MOF. MOF-CJ3 has the topology [M...,0.118663,:
3,MOF-CJ3,hasTopology,sab,MOF-CJ3 is an MOF. MOF-CJ3 has the topology [M...,0.103806,follows
4,MOF-CJ3,hasTopology,sab,MOF-CJ3 is an MOF. MOF-CJ3 has the topology [M...,0.037848,domain
...,...,...,...,...,...,...
995,DUT-63,hasTopology,tbo,DUT-63 is an MOF. DUT-63 has the topology [MASK].,0.000439,pictured
996,DUT-63,hasTopology,tbo,DUT-63 is an MOF. DUT-63 has the topology [MASK].,0.000439,the
997,DUT-63,hasTopology,tbo,DUT-63 is an MOF. DUT-63 has the topology [MASK].,0.000438,thus
998,DUT-63,hasTopology,tbo,DUT-63 is an MOF. DUT-63 has the topology [MASK].,0.000424,T


### Evaluate on Small

In [61]:
predicted_df.loc[10]['ObjectEntity'].split(",")

['sab']

In [65]:
predicted_df['predicted'] = predicted_df.apply(hasPredicted, axis=1)

In [66]:
predicted_df.head()

,SubjectEntity,Relation,ObjectEntity,prompt,predictedScore,predictedToken,predicted
0,MOF-CJ3,hasTopology,sab,MOF-CJ3 is an MOF. MOF-CJ3 has the topology [M...,0.239902,following,0
1,MOF-CJ3,hasTopology,sab,MOF-CJ3 is an MOF. MOF-CJ3 has the topology [M...,0.133175,of,0
2,MOF-CJ3,hasTopology,sab,MOF-CJ3 is an MOF. MOF-CJ3 has the topology [M...,0.118663,:,0
3,MOF-CJ3,hasTopology,sab,MOF-CJ3 is an MOF. MOF-CJ3 has the topology [M...,0.103806,follows,0
4,MOF-CJ3,hasTopology,sab,MOF-CJ3 is an MOF. MOF-CJ3 has the topology [M...,0.037848,domain,0


In [67]:
predicted_df['predicted'].sum()

0

In [68]:
predicted_df[predicted_df.predicted == 1]

,SubjectEntity,Relation,ObjectEntity,prompt,predictedScore,predictedToken,predicted


In [69]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [70]:
topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=100)

In [71]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [72]:
topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')

In [73]:
topk_agg_df.columns = ['SubjectEntity', 'predicted']
topk_agg_df

,SubjectEntity,predicted
0,DUT-63,0
1,MOF-1,0
2,MOF-2,0
3,MOF-CJ3,0
4,NJU-Bai9,0
5,UMCM-151,0
6,UMCM-152,0
7,URMOF-1,0
8,URMOF-2,0
9,"catena-(octakis(μ6-4,4',4''-s-Triazine-2,4,6-t...",0


In [74]:
topk_agg_df.predicted.mean()

0.0

## Test BERT

In [12]:
def prompt_probe(model, tokenizer, prompt_processor, df_sub, mask_token, \
                 subjectCol, relationCol, objectCol, top_k=100):

    pipe = pipeline(
        task="fill-mask",
        model=model,
        tokenizer=tokenizer,
        top_k=top_k,
    )

    prompts = PromptSet([prompt_processor(row.SubjectEntity, mask_token) for _, row in df_sub.iterrows()])

    outputs = []
    for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
        outputs.append(out)

    results = []
    for idx, row in df_sub.iterrows():
        for output in outputs[idx]:
            record = {}
            record['SubjectEntity'] = row[subjectCol]
            record['Relation'] = row[relationCol]
            record['ObjectEntity'] = row[objectCol]
            record['prompt']= prompts[idx]
            record['predictedScore'] = output['score']
            record['predictedToken'] = output['token_str']
            results.append(record)

    return pd.DataFrame(results)

In [13]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [14]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [15]:
# Assume the predicted_df has a binary column 'predicted' indicating whether ObjectEntity has been predicted
# 1 for yes, 0 for no
def evaluate_acc_n(predicted_df, relation, prompt_template, at_k=5):
    topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=at_k)
    topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')
    topk_agg_df.columns = ['SubjectEntity', 'predicted']
    result = {}
    result['Relation'] = relation
    result['prompt_template'] = prompt_template
    result['acc'] = topk_agg_df.predicted.mean()
    result['at_k'] = at_k

    return result

In [16]:
pd.set_option('display.max_colwidth', None)

In [17]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

...

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

CPU times: user 2h 12min 39s, sys: 3min 19s, total: 2h 15min 58s
Wall time: 10min 33s


In [18]:
hasType_df = pd.DataFrame(acc_list)

In [19]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
2,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.00000,10
31,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.00000,5
30,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.00000,1
21,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.00000,20
62,hasMetals,{SubjectEntity} is a MOF. The RCSR code of {SubjectEntity} is [MASK].,0.00000,10
22,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.00000,50
7,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.00000,5
29,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the three letter symbol [MASK].,0.01072,100
3,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.00000,20
49,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.00000,5


In [20]:
result_df.to_csv("../data/predicted_hasTopology_BERT.csv", index=None)

In [21]:
hasType_df.to_csv("../data/predicted_hasTopology_BERT_eval.csv", index=None)

In [22]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,1
6,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.0,1
12,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an tolology RCSR code [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.0,1
24,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the three letter symbol [MASK].,0.0,1
30,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.0,1
36,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.0,1
42,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is an MOF. The three letter [MASK] describes {SubjectEntity} topology.,0.0,1


In [23]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
41,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.029096,100
47,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.024502,100
40,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.019908,50
65,hasMetals,{SubjectEntity} is a MOF. The RCSR code of {SubjectEntity} is [MASK].,0.016845,100
39,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.012251,20
...,...,...,...,...
32,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,10
33,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,20
34,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,50
35,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,100


## Probing MatBERT

In [24]:
# MatBERT
tokenizer = AutoTokenizer.from_pretrained("../model/matbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("../model/matbert-base-cased")

In [25]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [26]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [27]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

...

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

CPU times: user 45min 17s, sys: 55.6 s, total: 46min 13s
Wall time: 4min 7s


In [28]:
hasType_df = pd.DataFrame(acc_list)

In [29]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
41,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.027565,100
40,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.026034,50
52,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.052067,50
62,hasMetals,{SubjectEntity} is a MOF. The RCSR code of {SubjectEntity} is [MASK].,0.000000,10
56,hasMetals,{SubjectEntity} is an MOF. The three letter [MASK] describes {SubjectEntity} topology.,0.000000,10
45,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.012251,20
66,hasMetals,"As a MOF, {SubjectEntity} has a topological structure [MASK].",0.000000,1
10,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.001531,50
6,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.000000,1
7,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.000000,5


In [30]:
result_df.to_csv("../data/predicted_hasTopology_MatBERT.csv", index=None)

In [31]:
hasType_df.to_csv("../data/predicted_hasTopology_MatBERT_eval.csv", index=None)

In [32]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,1
6,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.0,1
12,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an tolology RCSR code [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.0,1
24,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the three letter symbol [MASK].,0.0,1
30,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.0,1
36,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.0,1
42,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is an MOF. The three letter [MASK] describes {SubjectEntity} topology.,0.0,1


In [33]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
53,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.064319,100
52,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.052067,50
35,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.038285,100
41,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.027565,100
47,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.026034,100
...,...,...,...,...
30,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,1
31,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,5
32,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,10
33,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,20


## Probing MatSciBERT

In [34]:
# MatSciBERT
tokenizer = AutoTokenizer.from_pretrained('m3rg-iitd/matscibert')
model = AutoModelForMaskedLM.from_pretrained('m3rg-iitd/matscibert')

In [35]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [36]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [37]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

...

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

CPU times: user 46min 54s, sys: 43.3 s, total: 47min 37s
Wall time: 4min 14s


In [38]:
hasType_df = pd.DataFrame(acc_list)

In [39]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
7,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.0,5
1,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,5
6,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.0,1
53,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,100
0,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,1
66,hasMetals,"As a MOF, {SubjectEntity} has a topological structure [MASK].",0.0,1
55,hasMetals,{SubjectEntity} is an MOF. The three letter [MASK] describes {SubjectEntity} topology.,0.0,5
12,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an tolology RCSR code [MASK].,0.0,1
36,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.0,1
3,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,20


In [40]:
result_df.to_csv("../data/predicted_hasTopology_MatSciBERT.csv", index=None)

In [41]:
hasType_df.to_csv("../data/predicted_hasTopology_MatSciBERT_eval.csv", index=None)

In [42]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,1
6,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.0,1
12,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an tolology RCSR code [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.0,1
24,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the three letter symbol [MASK].,0.0,1
30,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.0,1
36,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.0,1
42,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is an MOF. The three letter [MASK] describes {SubjectEntity} topology.,0.0,1


In [43]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
23,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.024502,100
41,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.022971,100
47,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.019908,100
40,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.019908,50
46,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.016845,50
...,...,...,...,...
29,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the three letter symbol [MASK].,0.000000,100
30,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,1
31,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,5
32,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,10


## Probing SciBERT

In [44]:
# SciBERT
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModelForMaskedLM.from_pretrained('allenai/scibert_scivocab_cased')

In [45]:
mask_token = tokenizer.mask_token
mask_token

'[MASK]'

## Creat the Prompt Pipeline

In [46]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [47]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

...

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

CPU times: user 47min 16s, sys: 25.1 s, total: 47min 41s
Wall time: 4min 14s


In [48]:
hasType_df = pd.DataFrame(acc_list)

In [49]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
59,hasMetals,{SubjectEntity} is an MOF. The three letter [MASK] describes {SubjectEntity} topology.,0.003063,100
51,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.000000,20
67,hasMetals,"As a MOF, {SubjectEntity} has a topological structure [MASK].",0.000000,5
15,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an tolology RCSR code [MASK].,0.000000,20
22,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.000000,50
49,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.000000,5
60,hasMetals,{SubjectEntity} is a MOF. The RCSR code of {SubjectEntity} is [MASK].,0.000000,1
66,hasMetals,"As a MOF, {SubjectEntity} has a topological structure [MASK].",0.000000,1
71,hasMetals,"As a MOF, {SubjectEntity} has a topological structure [MASK].",0.000000,100
21,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.000000,20


In [50]:
result_df.to_csv("../data/predicted_hasTopology_SciBERT.csv", index=None)

In [51]:
hasType_df.to_csv("../data/predicted_hasTopology_SciBERT_eval.csv", index=None)

In [52]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,1
6,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.0,1
12,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an tolology RCSR code [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.0,1
24,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the three letter symbol [MASK].,0.0,1
30,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.0,1
36,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.0,1
42,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is an MOF. The three letter [MASK] describes {SubjectEntity} topology.,0.0,1


In [53]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
40,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.018377,50
41,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.018377,100
47,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.018377,100
46,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.018377,50
39,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.016845,20
...,...,...,...,...
31,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,5
32,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,10
33,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,20
34,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.000000,50


## Probing RoBERTa

In [54]:
# RoBERTa
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
model = AutoModelForMaskedLM.from_pretrained("roberta-large")

In [55]:
mask_token = tokenizer.mask_token
mask_token

'<mask>'

## Creat the Prompt Pipeline

In [56]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [57]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntity', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

...

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

  0%|          | 0/748 [00:00<?, ?it/s]

CPU times: user 2h 9min 9s, sys: 2.56 s, total: 2h 9min 12s
Wall time: 10min 1s


In [58]:
hasType_df = pd.DataFrame(acc_list)

In [59]:
hasType_df.sample(10)

,Relation,prompt_template,acc,at_k
51,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,20
55,hasMetals,{SubjectEntity} is an MOF. The three letter [MASK] describes {SubjectEntity} topology.,0.0,5
4,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,50
64,hasMetals,{SubjectEntity} is a MOF. The RCSR code of {SubjectEntity} is [MASK].,0.0,50
30,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.0,1
37,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.0,5
66,hasMetals,"As a MOF, {SubjectEntity} has a topological structure [MASK].",0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,1
10,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.0,50
26,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the three letter symbol [MASK].,0.0,10


In [60]:
result_df.to_csv("../data/predicted_hasTopology_RoBERTa.csv", index=None)

In [61]:
hasType_df.to_csv("../data/predicted_hasTopology_RoBERTa_eval.csv", index=None)

In [62]:
hasType_df[hasType_df['at_k'] == 1].sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,1
6,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an topology code [MASK].,0.0,1
12,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has an tolology RCSR code [MASK].,0.0,1
18,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.0,1
24,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the three letter symbol [MASK].,0.0,1
30,hasMetals,{SubjectEntity} is an metal organic framework. {SubjectEntity}'s topology is [MASK].,0.0,1
36,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is an type of [MASK] topology.,0.0,1
42,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} is in the class of [MASK] topology.,0.0,1
48,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,1
54,hasMetals,{SubjectEntity} is an MOF. The three letter [MASK] describes {SubjectEntity} topology.,0.0,1


In [63]:
hasType_df.sort_values('acc', ascending=False)

,Relation,prompt_template,acc,at_k
0,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,1
1,hasMetals,{SubjectEntity} is an MOF. {SubjectEntity} has the topology [MASK].,0.0,5
52,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,50
51,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,20
50,hasMetals,{SubjectEntity} has SBUs and organic linkers. The topology of {SubjectEntity} is [MASK].,0.0,10
...,...,...,...,...
23,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.0,100
22,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.0,50
21,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.0,20
20,hasMetals,{SubjectEntity} is an MOF. The topology of {SubjectEntity} is [MASK].,0.0,10
